# Plotting

We will plot with 3 datasets this week. Let's load them. 

In [1]:
import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_datareader as pdr  # IF NECESSARY, from terminal: pip install pandas_datareader
import seaborn as sns
from numpy.random import default_rng

# these three are used to open the CCM dataset:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

pd.set_option("display.max_rows", 10)  # display option for pandas
# more here: https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html

## Load macro_data

In [ ]:
# LOAD DATA AND CONVERT TO ANNUAL

start = 1990 # pandas datareader can infer these are years
end = 2018
macro_data = pdr.data.DataReader(['CAUR','MIUR','PAUR', # unemployment 
                                  'LXXRSA','DEXRSA','WDXRSA', # case shiller index in LA, Detroit, DC (no PA  available!)
                                  'MEHOINUSCAA672N','MEHOINUSMIA672N','MEHOINUSPAA672N'], #  
                                 'fred', start, end)
macro_data = macro_data.resample('Y').first() # get's the first observation for each variable in a given year

# CLEAN UP THE FORMATING SOMEWHAT

macro_data.index = macro_data.index.year
macro_data.columns=pd.MultiIndex.from_tuples([
    ('Unemployment','CA'),('Unemployment','MI'),('Unemployment','PA'),
    ('HouseIdx','CA'),('HouseIdx','MI'),('HouseIdx','PA'),
    ('MedIncome','CA'),('MedIncome','MI'),('MedIncome','PA')
    ])



In [ ]:
year_state_tall = macro_data.stack().reset_index().rename(columns={'level_1':'state'}).sort_values(['state','DATE'])    

year_state_wide = macro_data
# one level names
year_state_wide.columns=[
    'Unemployment_CA','Unemployment_MI','Unemployment_PA',
    'HouseIdx_CA','HouseIdx_MI','HouseIdx_PA',
    'MedIncome_CA','MedIncome_MI','MedIncome_PA'
    ]

## And load CCM data

First, load the data

In [2]:
url = 'https://github.com/LeDataSciFi/ledatascifi-2025/blob/main/data/CCM_cleaned_for_class.zip?raw=true'

#firms = pd.read_stata(url)   
# <-- that code would work, but GH said it was too big and
# forced me to zip it, so here is the work around to download it:

with urlopen(url) as request:
    data = BytesIO(request.read())

with ZipFile(data) as archive:
    with archive.open(archive.namelist()[0]) as stata:
        ccm = pd.read_stata(stata)

## Sidebar: Here's a fun EDA hack:

https://ydata-profiling.ydata.ai/docs/master/index.html

Notes
- Slow with huge datasets (see webpage 
- Doesn't work with multiindex column names (must be "one level")

In [ ]:
# install new package (run this one time only)
# !pip install -U ydata-profiling

# i got an error towards the end but it still installed...

In [ ]:
# this package used to be called pandas-profiling
# new name, better performance (speed and features)
# from ydata_profiling import ProfileReport

# create the report:
# profile = ProfileReport(macro_data, title="Pandas Profiling Report")
# profile.to_file("macro_data_report.html")



From the `year_state` data (wide or tall):

_("unemployment changes": Focus on the change in the _level_ (raw difference, not percent change) of unemployment from one year compared the prior year.)_

- Q0. How has median income has evolved over time for PA?
    - Demos...
- Q1. How has *unemployment changes* evolved over time for PA?   
- Q2. What is the distribution of unemployment changes for all states (view as one var)?   
- Q3. What is the distribution of unemployment changes for all states (separately)?
- Q4. How does unemployment changes vary with median income growth?

From the `ccm` data:

- Q5. Plot the distribution of R&D (`xrd_a`). Bonuses:
    - deal with outliers
    - add a title
    - change the x and y axis titles
- Q6: Compare R&D and CAPX. Bonuses:
    - don't plot outliers
    - avoid oversaturated plot

In [ ]:
year_state_tall # aka tidy data in the "R" world - a column is a variable, a row has a key uniquely IDing it

In [ ]:
year_state_wide

In [ ]:
# - Q0. How has median income has evolved over time for PA?
#     - Demos...

In [ ]:
# from the WIDE, using pandas plot
year_state_wide['MedIncome_PA'].plot()

In [ ]:
# from the tall, using pandas
med_income_pa = year_state_tall.query('state=="PA"')[['DATE','MedIncome']]

med_income_pa.set_index('DATE').plot()  


In [ ]:
# from the tall, using pandas (one line)
year_state_tall.query('state=="PA"')[['DATE','MedIncome']].set_index('DATE').plot()  


In [ ]:
# from the tall, using seaborn

# plot median income for PA over time
sns.lineplot(data=year_state_tall.query('state=="PA"'), # you can query in the data arg 
             x='DATE', 
             y='MedIncome')

# one way to add labels and stuff - plt.thing('text')
plt.xlabel('Year')
plt.title('Median Income in PA')


In [ ]:
# tall with sns with alt query 
sns.lineplot(x='DATE',y='MedIncome',
             data=year_state_tall[ year_state_tall['state'].isin(['PA']) ] )

# year_state_tall['state'] == "PA"

In [ ]:
# by default, with tall and groups of obs, SNS will aggregate Y by X, and plot mean/error bars 
sns.lineplot(x='DATE',y='MedIncome',
             data=year_state_tall)


In [ ]:
# pandas wide
vars = ['MedIncome_CA','MedIncome_MI','MedIncome_PA']
year_state_wide[vars].plot()

In [ ]:
# from tall with seaborn, easy to replicate:
# hue controls grouping (row and col do too, but differently)_
sns.lineplot(x='DATE',y='MedIncome',
             data=year_state_tall, 
            hue='state')

# Q1 - WARNING/LESSON/ABCD

If you create a variable in a TALL dataset based on prior rows of data... groupby!

Else you propogate info from one state to next (one firm to next...)

In [ ]:
# on tuesday, we started with this:

year_state_wide['Unemployment_PA'].diff().plot()
# year_state_wide
# this works

In [ ]:
# the other way to do it is via the tall dataframe 
year_state_tall['Unemployment_DIFF'] = year_state_tall['Unemployment'].diff()

year_state_tall['diff'] = year_state_tall.groupby('state')['Unemployment'].diff()

year_state_tall # BUT THIS HAS A PROBLEM 
year_state_tall[24:34] # diff is missing for MI in 1990, as it should be!

## Two part lesson!

If you are making a variable based on some units, like "rolling firm returns" or "country level inflation"
1. ABCD - look at the data where you go from one unit to the next
2. Create those variables with a groupby! (if the data is "tall")

In [ ]:
# - Q1. How has *unemployment changes* evolved over time for PA?   
# year_state_tall['diff'] = year_state_tall.groupby('state')['Unemployment'].diff()

# # lets check!!!
# # two options: iloc/list slice ;  or variable inspector
# year_state_tall[25:35]

In [ ]:
# sns with the tall data
sns.lineplot(data=year_state_tall.query('state == "PA"' ),
             x = "DATE", 
             y = "diff")

In [ ]:
# if we remove the "query" that limits the data to just PA, it wants to plot all the states
# but what is the 1994 number to use? By default, it will avg them
# instead. use hue to say "diff lines for each state"
sns.lineplot(data=year_state_tall,
             x = "DATE", 
             y = "diff", hue='state')

In [ ]:
# q2

# you can modify seaborn plots via .set() 
(
sns
    .kdeplot(year_state_tall['diff'])    
    .set(title='Hey', xlabel='Some words')
)
plt.show() # optional, deletes the stupid msg

In [ ]:
# one q3 solution
sns.displot(kind='hist', col="state", x="diff", data=year_state_tall, stat='percent')

# we played around with lots of alts here

In [ ]:
#q4
year_state_tall['MedIncome_Change'] = year_state_tall.groupby('state')['MedIncome'].pct_change()


In [ ]:
year_state_tall
(
sns
.lmplot(x="MedIncome_Change", y="diff", data=year_state_tall, hue='state')
.set(ylabel='Change in Une Rate',xlabel='Inc Growth')
)

In [ ]:
# q5 - kyles code 

# calculate the iqr to find outliers  (in a box plot, the dots outside the whiskers are foudn via this same method)
Q1 = ccm['xrd_a'].quantile(0.25)
Q3 = ccm['xrd_a'].quantile(0.75)
IQR = Q3 - Q1

# define range for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# eliminate outliers
xrd_no_outliers = ccm[(ccm['xrd_a'] >= lower_bound) & (ccm['xrd_a'] <= upper_bound)]

# Plot distribution
xrd_no_outliers['xrd_a'].plot(kind = 'hist', bins=30, title = 'R&D Distribution')
plt.xlabel('R&D (xrd_a)')  # label x-axis
plt.ylabel('Frequency')    # label y-axis


In [ ]:
(
    # :1000 just so it plots faster 
    # trick+ you can modify the data in the data= spot
    # you can sample part of the data to plot with data=df.sample()
sns
    .kdeplot(data=ccm[:1000].query('xrd_a >=0 & xrd_a < .25'),
    )
)

In [ ]:

- Q5. Plot the distribution of R&D (`xrd_a`). Bonuses:
    - deal with outliers
    - add a title
    - change the x and y axis titles
- Q6: Compare R&D and CAPX. Bonuses:
    - don't plot outliers
    - avoid oversaturated plot

In [ ]:
sns.scatterplot(data=ccm,x='xrd_a',y='capx_a')